<font color='green'>***Installation and Libraries Import***</font>
---
--- 

In [ ]:

%pip install flwr
%pip install ray 
%pip install --upgrade pip
%pip install torch torchvision matplotlib
%pip install async-timeout
%pip install async-numpy
%pip install pandas
%pip install datasets
%pip install scikit-learn


In [ ]:
import os
import flwr as fl
import torch, ray, pandas, sklearn
import pickle
from pprint import pprint
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader


print("All modules loaded successfully!")
print("FLWR version:", fl.__version__)
print("Ray version:", ray.__version__)
print("Torch version:", torch.__version__)

libraries_to_uninstall = [
    "tb-nightly==2.18.0a20240701",
    "tensorboard==2.16.2",
    "tensorboard-data-server==0.7.2",
    "tensorboard-plugin-wit==1.8.1",
    "tensorflow==2.16.2",
    "tensorflow-io-gcs-filesystem==0.37.0",
    "termcolor==2.4.0",
    "terminado==0.18.1",
    "tf-estimator-nightly==2.8.0.dev2021122109",
    "tf_keras-nightly==2.18.0.dev2024070109",
    "tf-nightly==2.18.0.dev20240626"
]
for library in libraries_to_uninstall:
    os.system(f"pip uninstall -y {library}")
print("All modules loaded successfully!")
print("FLWR version:", fl.__version__)
print("Ray version:", ray.__version__)
print("Torch version:", torch.__version__)

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, random_split, Subset
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
import flwr as fl
from flwr.common import Metrics
# from flwr_datasets import FederatedDataset
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
import random
import math
import pickle
import csv
import copy
print(fl.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

<font color='Brown'>***Constants***</font>
---
--- 

In [ ]:
# DEVICE
NUM_CLIENTS = 48 #48
ROUNDS = 40 #40
BATCH_SIZE = 64 #C
LEARNING_RATE = 0.0025 #C
EPOCHS = 3 #C
DATA_GROUPS = 40
BATCH_ROUND = 6
SIZE_ROUND = int(BATCH_ROUND * BATCH_SIZE * NUM_CLIENTS)
NUM_ATCKS = 5
FL = True # FL or CENT
MODE = 'DNN' #DNN or TT
PATH = f'{MODE}-FL-{FL}-{NUM_CLIENTS}-clients-{NUM_ATCKS}-atk-{ROUNDS}-rounds-{EPOCHS}-epochs-{LEARNING_RATE}-lr-{DATA_GROUPS}-groups'
G = 0

<font color='Light Blue'>***Dataset Preparations***</font>
---
--- 

In [ ]:
TrafficData = {}
TrafficData['Dataset']={}
sets_names = ['30','100','70','50','120']
for  DATA_NUM in sets_names:
    TrafficData['Dataset'][DATA_NUM]=pd.read_csv(f'data/2_Dataset_{NUM_ATCKS}_Attack_{DATA_NUM}_normal.csv', low_memory=False, quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(DATA_NUM, TrafficData['Dataset'][DATA_NUM].shape)
for DATA_NUM in TrafficData['Dataset']:
    TrafficData['Dataset'][DATA_NUM]=TrafficData['Dataset'][DATA_NUM].sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
TrafficData['Split'] = {}
sets_training =  ['30','100','70','50']
for DATA_NUM in sets_training:
    TrafficData['Split'][DATA_NUM] = np.array_split(TrafficData['Dataset'][DATA_NUM],DATA_GROUPS)

TrafficData['Combined'] = pd.concat([TrafficData['Split']['30'][0], TrafficData['Split']['100'][0], TrafficData['Split']['70'][0], TrafficData['Split']['50'][0]]).reset_index(drop=True)
for GROUP in range(1, DATA_GROUPS):
    TrafficData['Combined'] = pd.concat([TrafficData['Combined'], TrafficData['Split']['30'][GROUP], TrafficData['Split']['100'][GROUP], TrafficData['Split']['70'][GROUP], TrafficData['Split']['50'][GROUP]]).reset_index(drop=True)
print(TrafficData['Combined'].shape)            

In [ ]:
TrafficData['Train'] = {}
TrafficData['Train']['X'] = TrafficData['Combined'].iloc[:, 0:-1]
TrafficData['Train']['y'] = TrafficData['Combined'].iloc[:, -1]
print(TrafficData['Train']['X'].shape)
print(TrafficData['Train']['y'].shape)

TrafficData['Test'] = {}
TrafficData['Test']['X']=TrafficData['Dataset']['120'].iloc[:, 0:-1]
TrafficData['Test']['y']=TrafficData['Dataset']['120'].iloc[:, -1]
print(TrafficData['Test']['X'].shape)
print(TrafficData['Test']['y'].shape)

In [ ]:
scaler = MinMaxScaler()
model = scaler.fit(TrafficData['Train']['X'])
TrafficData['Train']['X'] = model.transform(TrafficData['Train']['X'])
TrafficData['Test']['X'] = model.transform(TrafficData['Test']['X'])

TrafficData['Train']['X'], TrafficData['Train']['y']= np.array(TrafficData['Train']['X']), np.array(TrafficData['Train']['y'])
print(type(TrafficData['Train']['X']),type(TrafficData['Train']['y']))
print(TrafficData['Train']['X'].shape,TrafficData['Train']['y'].shape)
TrafficData['Test']['X'], TrafficData['Test']['y']= np.array(TrafficData['Test']['X']), np.array(TrafficData['Test']['y'])
print(type(TrafficData['Test']['X']),type(TrafficData['Test']['y']))
print(TrafficData['Test']['X'].shape,TrafficData['Test']['y'].shape)

In [ ]:
TrafficData['ROUNDS']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['ROUNDS'][ROUND]={}

SIZE_Demo = SIZE_ROUND
for ROUND in range(1,ROUNDS+1):
    if ROUND == 1:
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][:SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][:SIZE_Demo]
    else:
        print((SIZE_Demo - SIZE_ROUND),SIZE_Demo)
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
    SIZE_Demo = SIZE_Demo + SIZE_ROUND
for ROUND in TrafficData['ROUNDS']:
    print("ROUND: ", ROUND, TrafficData['ROUNDS'][ROUND]['X'].shape, TrafficData['ROUNDS'][ROUND]['y'].shape)
print(SIZE_Demo, SIZE_ROUND)

In [ ]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

In [ ]:
TrafficData['trainsets']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['trainsets'][ROUND]= ClassifierDataset(torch.from_numpy(TrafficData['ROUNDS'][ROUND]['X']).float(), torch.from_numpy(TrafficData['ROUNDS'][ROUND]['y']).long())
TrafficData['testset'] = ClassifierDataset(torch.from_numpy(TrafficData['Test']['X']).float(), torch.from_numpy(TrafficData['Test']['y']).long())

In [ ]:
def load_train(numberofclients, ROUND):    
    # Use the actual dataset length to compute balanced portions per client
    dataset_len = len(TrafficData['trainsets'][ROUND])
    if dataset_len == 0:
        # Return an empty list of loaders to avoid indexing errors downstream
        return [DataLoader(Subset(TrafficData['trainsets'][ROUND], []), batch_size=BATCH_SIZE, shuffle=False) for _ in range(numberofclients)]
    
    # Distribute samples as evenly as possible across clients (handle remainders)
    num_portions = int(numberofclients)
    base_portion = dataset_len // num_portions
    remainder = dataset_len % num_portions
    portion_indices = []
    start_idx = 0
    for i in range(num_portions):
        # First `remainder` clients receive one extra sample
        sz = base_portion + (1 if i < remainder else 0)
        end_idx = start_idx + sz
        if sz > 0:
            portion_indices.append(list(range(start_idx, end_idx)))
        else:
            portion_indices.append([])
        start_idx = end_idx
    
    # Build Subset and DataLoader for each client (safe: indices are within [0, dataset_len))
    portion_datasets = [Subset(TrafficData['trainsets'][ROUND], indices) for indices in portion_indices]
    portion_loaders = [DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False) for dataset in portion_datasets]            
    return portion_loaders
def load_test(numberofclients):    
    testloader = DataLoader(TrafficData['testset'], batch_size=BATCH_SIZE, shuffle=False)
    return testloader

In [ ]:
Dataloaders = {}
for ROUND in range(1, ROUNDS+1):
    Dataloaders[ROUND] = load_train(NUM_CLIENTS, ROUND)
Dataloaders['Test'] = load_test(NUM_CLIENTS)

In [ ]:
# Diagnostic: print per-client dataset sizes for the first few rounds to verify splits
for ROUND in range(1, min(6, ROUNDS+1)):
    loaders = Dataloaders.get(ROUND, None)
    if loaders is None:
        print(f"Dataloaders for Round {ROUND} not found (run the rebuild cell).")
        continue
    sizes = [len(loader.dataset) for loader in loaders]
    print(f"Round {ROUND}: per-client sizes (first 12): {sizes[:12]}")
    print(f"  Total samples this round: {sum(sizes)}\n")

In [ ]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

In [ ]:
for i, batch in enumerate(Dataloaders[5][0]):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

In [ ]:
from collections import Counter
for CLUSTER in range (1, 9):
    DEVICE_PERCENTAGE = []
    for DEVICE__ in range(0,NUM_CLIENTS):
        for i, batch in enumerate(Dataloaders[CLUSTER][DEVICE__]):
            _, labels = batch
            class_counts = Counter(labels.numpy())
            total_records = sum(class_counts.values())
            class_0_count = class_counts.get(0, 0)
            percentage_class_0 = (class_0_count / total_records) * 100
            DEVICE_PERCENTAGE.append(percentage_class_0)
            # print(f"Batch {i+1}: {dict(class_counts)}")
            # print(f"Percentage of class 0: {percentage_class_0:.2f}%\n")
    # print(DEVICE_PERCENTAGE)        
    chunk_size = 6
    averages = [sum(DEVICE_PERCENTAGE[i:i + chunk_size]) / chunk_size for i in range(0, len(DEVICE_PERCENTAGE), chunk_size)]
    # print("Averages of every device:")
    # print(averages)
    chunk_size_4 = 4
    averages = [sum(averages[i:i + chunk_size_4]) / chunk_size_4 for i in range(0, len(averages), chunk_size_4)]
    # print("Averages of every 4 devices:")
    print(averages)

In [ ]:
del TrafficData

<font color='Red'>***Neural Network***</font>
---
--- 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        self.layer_1 = nn.Linear(98, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 15) 
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_out(x)        
        return x

In [ ]:
# Random  = Net()
# for param_tensor in Random.state_dict():
#     print(param_tensor, "\t", Random.state_dict()[param_tensor].size())
# torch.save(Random.state_dict(), "0_Input_Random_model_Net.pth")

In [ ]:
def train(net, trainloader, epochs: int, verbose=True):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    net.train()
    prediction_matrix = []
    actual_matrix = []
    acc_matrix = []
    loss_matrix = []
    # Safeguard: if the trainloader has no samples, skip training
    try:
        dataset_len = len(trainloader.dataset)
    except Exception:
        dataset_len = 0
    if dataset_len == 0:
        if verbose:
            print("Warning: trainloader has 0 samples, skipping training.")
        return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Accumulate loss as scalar sum over samples
            epoch_loss += loss.item() * labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            predictions = torch.max(outputs.data, 1)[1]
            prediction_matrix.append(predictions.tolist())
            actual_matrix.append(labels.tolist())
        # Avoid division by zero if total==0 for safety
        if total == 0:
            epoch_loss_val = 0.0
            epoch_acc = 0.0
        else:
            epoch_loss_val = epoch_loss / total
            epoch_acc = correct / total
        loss_matrix.append(epoch_loss_val)
        acc_matrix.append(epoch_acc)
        if verbose:
            print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss_val:.6f} | Acc: {epoch_acc:.4f}")
    return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
def test(net, testloader):
    criterion = nn.CrossEntropyLoss()
    correct, total, loss_sum = 0, 0, 0.0
    net.eval()
    prediction_matrix = []
    actual_matrix = []
    acc_matrix = []
    loss_matrix = []
    try:
        test_len = len(testloader.dataset)
    except Exception:
        test_len = 0
    if test_len == 0:
        print("Warning: testloader has 0 samples, skipping evaluation.")
        return 0.0, 0.0, prediction_matrix, actual_matrix, acc_matrix, loss_matrix
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss_sum += criterion(outputs, labels).item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    if total == 0:
        avg_loss = 0.0
        accuracy = 0.0
    else:
        avg_loss = loss_sum / total
        accuracy = correct / total
    loss_matrix.append(avg_loss)
    acc_matrix.append(accuracy)
    print(f"Evaluation: eval loss {avg_loss}, eval accuracy {accuracy}")
    return avg_loss, accuracy, prediction_matrix, actual_matrix, acc_matrix, loss_matrix

In [ ]:
prediction_dict= {}
actual_dict= {}
accuracy_dict= {}
loss_dict= {}